In [ ]:
!pip install garden_ai

In [2]:
from garden_ai import GardenClient

In [ ]:
gc = GardenClient()

In [4]:
wisconsin_garden = gc.get_published_garden("10.23677/9j8g-6g51")

In [5]:
compounds = ['La0.6Sr0.4Co0.2Fe0.8O3', 'Ba0.5Sr0.5Co0.8Fe0.2O3', 'La0.8Sr0.2MnO3']
electrolytes = ['ceria', 'ceria', 'zirconia']

In [7]:
wisconsin_garden.predict_asr(compounds, electrolytes)

,Compositions,Electrolytes,Cost ($/kg),Stability @ 500C (meV/atom),ASR barrier (eV),log ASR at 500C (Ohm-cm2),log ASR error (Ohm-cm2)
0,La0.6Sr0.4Co0.2Fe0.8O3,ceria,133.672989,41.669610,1.411684,1.326496,0.471312
1,Ba0.5Sr0.5Co0.8Fe0.2O3,ceria,8.750681,130.604608,1.205874,0.206754,0.553044
2,La0.8Sr0.2MnO3,zirconia,169.046056,214.202900,1.564999,3.552795,0.575237
